In [21]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

<img src="images/hw_boston2.jpg">

# Добро пожаловать!

Зайдите на http://www.kaggle.com и зарегистрируйтесь. 

Далее вам нужен датасет по этой ссылке: https://www.kaggle.com/c/ghouls-goblins-and-ghosts-boo . И __train__, и __test__, и __sample_submission__. 

Скачайте его и положите рядом с тетрадью, как вам удобнее.

In [22]:
train, test = pd.read_csv(
    # путь к вашему файлу train
), pd.read_csv( 
    # путь к вашему файлу test
)

In [23]:
train.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost


In [24]:
X = train.drop(['id', 'type'], axis=1)
y = train['type']

# К делу!

## Выполним необходимое кодирование

В этот раз я помогу немного :)

### Метки классов

In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
answers_encoder = LabelEncoder()
y = answers_encoder.fit_transform(y)

In [27]:
y[:5] # вместо строк получились метки классов

array([1, 2, 1, 1, 0])

In [28]:
answers_encoder.classes_ # - здесь наши классы все еще лежат

array(['Ghost', 'Ghoul', 'Goblin'], dtype=object)

Используйте метод __inverse_transform__, чтобы превратить предсказания алгоритмов (числа) обратно в строки:

In [29]:
from sklearn.linear_model import LogisticRegression

In [31]:
answers_encoder.inverse_transform(
    # сюда массив с предсказаниями
)

```python
# типа того:
clf = LogisticRegression()
clf.fit(X_train, y_train);
predicts = clf.predict(X_test)
strings = answers_encoder.inverse_transform(predicts) # <-- !
```

### Теперь все признаки

У нас встретился признак цвета - __color__ - закодируем его так, как я сделал ниже. В этот раз.

In [32]:
def onehot_encode(df_train, df_test, column):
    from sklearn.preprocessing import LabelBinarizer
    
    cs = df_train.select_dtypes(include=['O']).columns.values
    if column not in cs:
        return (df_train, df_test, None)

    rest = [x for x in df_train.columns.values if x != column]

    lb = LabelBinarizer()
    train_data = lb.fit_transform(df_train[column])
    test_data = lb.transform(df_test[column])
    new_col_names = ['%s_%s' % (column, x) for x in lb.classes_]

    if len(new_col_names) != train_data.shape[1]:
        new_col_names = new_col_names[::-1][:train_data.shape[1]]

    new_train = pd.concat((df_train.drop([column], axis=1), pd.DataFrame(data=train_data, columns=new_col_names)), axis=1)
    new_test = pd.concat((df_test.drop([column], axis=1), pd.DataFrame(data=test_data, columns=new_col_names)), axis=1)
    return (new_train, new_test, lb)

In [33]:
X.head(2)

,bone_length,rotting_flesh,hair_length,has_soul,color
0,0.354512,0.350839,0.465761,0.781142,clear
1,0.575560,0.425868,0.531401,0.439899,green


In [34]:
X, test, lb = onehot_encode(X, test, 'color')

In [35]:
X.head(2)

,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white
0,0.354512,0.350839,0.465761,0.781142,0,0,0,1,0,0
1,0.575560,0.425868,0.531401,0.439899,0,0,0,0,1,0


In [36]:
test.head(2) 

,id,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white
0,3,0.471774,0.387937,0.706087,0.698537,1,0,0,0,0,0
1,6,0.427332,0.645024,0.565558,0.451462,0,0,0,0,0,1


Мы превратили все признаки в числовой вид, и с этим можно работать!

# Визуализация

Исследуйте датасет. Нарисуйте что-нибудь. Go!

_P.s._: не забывайте, что оригинальные данные все еще лежат в __train__.

# Классификация

Зафиксируем генератор случайных чисел, чтобы у всех вас получались одни и те же значения:

In [37]:
SEED = 1234
np.random.seed = SEED

Сформируем отложенную выборку:

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.85, random_state=SEED) # <- проверяйте у себя:
                                                                                        # если что-то принимает random_state 
                                                                                        # параметром, передавайте ему SEED

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 1.</h3> 
    </div>
    <div class="panel">
        Обучите <b>LogisticRegression</b> (конечно же, с лучшими найденными параметрами с помощью GridSearch) из пакета <b>sklearn.linear_model</b>. Проверьте его качество.
    </div>
</div>

In [ ]:
# Ваш код здесь

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 2.</h3> 
    </div>
    <div class="panel">
        Время дать шанс <b>KNearestNeighbors</b> из пакета <b>sklearn.neighbors</b> и <b>SVC</b> из пакета <b>sklearn.svm</b>. Сделайте то же, что и сделали выше.
    </div>
</div>

In [ ]:
# Ваш код здесь

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 3.</h3> 
    </div>
    <div class="panel">
        Проанализируйте лучшие классификаторы:
        <ol>
            <li> Распечатайте отчет о классификации (<b>classification report</b> из <b>sklearn.metrics</b>)
            <li> Постройте ROC-AUC-кривую (с помощью scikit-plot)
            <li> Постройте матрицу перепутывания классов (с помощью scikit-plot)
        </ol>
    </div>
</div>

In [ ]:
# Ваш код здесь

### Classification report

In [ ]:
# Ваш код здесь

### ROC-AUC

In [ ]:
# Ваш код здесь

### Classification report

In [ ]:
# Ваш код здесь

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 4.</h3> 
    </div>
    <div class="panel">
        Предскажите найденным вами лучшим алгоритмом тестовую (датафрейм <b>test</b>) выборку. Обратите внимание на <b>sample_submission</b> - формат отправки ответов. Сделайте так же, например.
        <br><br>
        Отошлите результаты на Kaggle в их проверяющую систему.
        <br><br>
        Расскажите о ваших результатах в Slack :)
    </div>
</div>

In [ ]:
# Ваш код здесь

С помощью кода ниже можно записать ваши предсказания в файл:

```python
answers = pd.DataFrame(data={'id': [1,2,3], 'type': ['Ghost','Goblin','Ghoul']})
answers.to_csv('submission.csv', index=False)
```

<img src="images/hw_boston3.jpg">